In [6]:
import numpy as np
import pandas as pd
import psycopg2

In [7]:
csv_file = "updated_b.csv"
df_plants = pd.read_csv(csv_file,error_bad_lines=False)
csv_file = "socio.csv"
df_pop = pd.read_csv(csv_file,error_bad_lines=False)
csv_file = "CO2.csv"
df_co2 = pd.read_csv(csv_file,error_bad_lines=False)

In [3]:
df_group = df_pop.groupby('Country', as_index=False).last()
df_group.drop(columns= 'GDP-PPP (Million Intl$ (2011))').join(df_plants)

,Country,Year,Population (People),GDP-USD (Million US$ (2010)),country_long,name,capacity_mw,latitude,longitude,primary_fuel,commissioning_year,owner,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Afghanistan,2014.0,3.275802e+07,1.999032e+04,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,1.00,34.6410,69.7170,Hydro,1000,state,NaN,NaN,NaN
1,Albania,2014.0,2.889104e+06,1.276823e+04,Afghanistan,Nangarhar (Darunta) Hydroelectric Power Plant ...,11.55,34.4847,70.3633,Hydro,1000,state,NaN,NaN,NaN
2,Algeria,2014.0,3.911331e+07,1.828894e+05,Afghanistan,Sarobi Dam Hydroelectric Power Plant Afghanistan,0.22,34.5865,69.7757,Hydro,1000,state,NaN,NaN,NaN
3,Andorra,2014.0,7.922300e+04,3.127736e+03,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,0.33,32.3220,65.1190,Hydro,1000,state,NaN,NaN,NaN
4,Angola,2014.0,2.692047e+07,1.008863e+05,Afghanistan,Northwest Kabul Power Plant Afghanistan,0.42,34.5638,69.1134,Gas,1000,state,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Vietnam,2014.0,9.072890e+07,1.448347e+05,Argentina,C.T. SAN VICENTE,27.60,-35.0241,-58.4237,Oil,1000,ENARSA,NaN,NaN,NaN
189,World,2014.0,7.268986e+09,7.347854e+07,Argentina,PARQUE EOLICO RAWSON II (ENARSA),28.80,-43.3522,-65.1734,Wind,1000,ENARSA,NaN,NaN,NaN
190,Yemen,2014.0,2.624633e+07,2.890029e+04,Argentina,CENTRAL TERMICA BRIGADIER LOPEZ (ENARSA),2.80,-31.7112,-60.7811,Gas,2013,ENARSA,NaN,NaN,NaN
191,Zambia,2014.0,1.562097e+07,2.531884e+04,Argentina,H. PICHI PICUN LEUFU,2.85,-40.0120,-69.9918,Hydro,1999,PETROBRAS ENERGIA,NaN,NaN,NaN


In [5]:
df_co2_group = df_co2.groupby('Country').last()
column_select = ['country_long',
                 'Population (People)',
                 'name',
                 'capacity_mw',
                 'primary_fuel',
                 'commissioning_year',
                 'Country','latitude',
                 'longitude',
                 'GDP-USD (Million US$ (2010))']
df_plants_pop = df_plants.merge(df_group, 
                                how='outer' ,
                                left_on ='country_long', 
                                right_on = 'Country' )[column_select].dropna()

In [8]:
column_select = ['country_long',
                 'name',
                 'capacity_mw',
                 'commissioning_year',
                 'primary_fuel','Country',
                 'latitude','longitude',
                 'Population (People)',
                 'GDP-USD (Million US$ (2010))',
                 'Total CO2 Emissions Excluding Land-Use Change and Forestry (MtCO2)']

In [10]:
df_merge = df_plants_pop.merge(df_co2_group ,
                               how='outer' ,
                               left_on ='Country',
                               right_on = 'Country' )[column_select].dropna()

In [12]:
df_final = df_merge.drop('Country', axis = 1)

In [13]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connected to the PostgreSQL database')
        conn = psycopg2.connect(**params_dic)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return conn

In [14]:
""" Execute a single INSERT request """
def single_insert(conn, insert_req): 
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [26]:
def connect_db(host, db, user, password, sslmode):
    try:
        connection = psycopg2.connect(host=host, database=db,user=user,password=password, sslmode=sslmode)
        print(f'Logged in {db} successfully !')
    except e:
        print(f'Could not log in to {db} !')
    return connection

In [27]:
conn = connect_db("dtv.postgres.database.azure.com",
                  "plants2",
                  "energy@dtv.postgres.database.azure.com",
                  "ecolo123!",
                  "require")
cursor = conn.cursor()
conn.autocommit = True

Logged in plants2 successfully !


In [18]:
"""
0 country_long 
1 name 
2 capacity_mw 
3 commissioning_year 
4 primary_fuel 
5 latitude 
6 longitud
7 Population (People) 
8 GDP-USD (Million US$ (2010))
9 Total CO2 Emissions Excluding Land-Use Change and Forestry (MtCO2)
"""

'\n0 country_long \n1 name \n2 capacity_mw \n3 commissioning_year \n4 primary_fuel \n5 latitude \n6 longitud\n7 Population (People) \n8 GDP-USD (Million US$ (2010))\n9 Total CO2 Emissions Excluding Land-Use Change and Forestry (MtCO2)\n'

In [25]:
"""psycopg2.connect(host="dtv.postgres.database.azure.com",
                 database="plants2",
                 user="energy@dtv.postgres.database.azure.com",
                 password="ecolo123!",
                 sslmode = "require")"""

<connection object at 0x7f2adf1167d0; dsn: 'user=energy@dtv.postgres.database.azure.com password=xxx dbname=plants2 host=dtv.postgres.database.azure.com sslmode=require', closed: 0>

In [28]:
for central_list in df_final.values:
    
    country_name = central_list[0]
    pp_name = central_list[1]
    capacity = central_list[2]
    commissioning_year = central_list[3]
    fuel = central_list[4]
    latitude = central_list[5]
    longitude = central_list[6]
    population = central_list[7]
    gdp = central_list[8]
    co2 = central_list[9]
    
    query = """
    INSERT into countries(name) values('%s')
    ON CONFLICT DO NOTHING;
    """ % (country_name)
    single_insert(conn, query)
    
    cursor.execute("SELECT id FROM countries WHERE name = %s", (country_name,))
    id_country = cursor.fetchone()[0]
    
    query = """
    INSERT into power_plants(name,capacity, commissioning_year, lat,long,fuel,id_country ) values('%s','%s','%s','%s','%s','%s','%s');
    """ % (pp_name, capacity, commissioning_year, latitude, longitude, fuel, id_country)
    single_insert(conn, query)
    
    query = """
    INSERT into economics(population,gdp_usd,id_country ) values('%s','%s','%s');
    """ % (population, gdp, id_country)
    single_insert(conn, query)
    
    query = """
    INSERT into emissions(co2, id_country) values('%s', '%s');
    """ % (co2, id_country)
    single_insert(conn, query)